In [81]:
# from google.colab import drive
# drive.mount('/content/drive')

In [82]:
import transformers
import math
import statistics
from tqdm import tqdm
import torch.nn.functional as F
import torch
import os
import numpy as np
import pandas as pd
import seaborn as sns
from transformers import pipeline
from torch.utils.data.dataloader import default_collate
# !pip install torchvision
from torchvision.models import resnet101
from torchvision.models import resnet50
from torch.utils.data import DataLoader
import torch
import av
import torchvision
from torch import nn, optim
from torchvision import transforms
from torchvision.datasets import UCF101


In [84]:
frames_per_clip = 5
step_between_clips = 1
batch_size = 128
path='../data/ucfTrainTestlist'

In [87]:
ucf_data_dir = "../data/UCF-101/"
ucf_label_dir = path
frames_per_clip = 5
step_between_clips = 1
batch_size = 128

In [88]:
def divide_by_255(x):
    return x / 255.

def permute_channels(x):
    return x.permute(0, 3, 1, 2)

def interpolate(x):
    return nn.functional.interpolate(x, (240, 320))

tfs = transforms.Compose([
    transforms.Lambda(divide_by_255),
    transforms.Lambda(permute_channels),
    transforms.Lambda(interpolate),
])

def custom_collate(batch):
    filtered_batch = []
    for video, _, label in batch:
        filtered_batch.append((video, label))
    return torch.utils.data.dataloader.default_collate(filtered_batch)


In [89]:
# load train test datasets
train_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips, train=True, transform=tfs)

test_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                      step_between_clips=step_between_clips, train=False, transform=tfs)

100%|██████████| 833/833 [03:01<00:00,  4.60it/s]


In [91]:
# Load train_dataset

# Load train_dataset_100k
with open('../data/train_dataset_100k.pt', 'rb') as f:
    train_set = torch.load(f)

# Load test_dataset_20k
with open('../data/test_dataset_20k.pt', 'rb') as f:
    test_set = torch.load(f)

# Load train_dataset_100k_rest
with open('../data/train_dataset_100k_rest.pt', 'rb') as f:
    test_set_org = torch.load(f)

# Load test_dataset_20k_rest
with open('../data/test_dataset_20k_rest.pt', 'rb') as f:
    test_set2 = torch.load(f)
    
train_loader_100k = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)
test_loader_20k = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=True,
                                          collate_fn=custom_collate)

# with open('../data/train_dataset.pt', 'rb') as f:
#     train_dataset = torch.load(f)

# # Load test_dataset
# with open('../data/test_dataset.pt', 'rb') as f:
#     test_dataset = torch.load(f)

# train_loader_full = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
#                                            collate_fn=custom_collate)
# test_loader_full = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True,
#                                           collate_fn=custom_collate)

In [92]:
print(f"Total number of train samples: {len(train_dataset)}")
print(f"Total number of test samples: {len(test_dataset)}")
print(f"Total number of (train) batches: {len(train_loader_100k)}")
print(f"Total number of (test) batches: {len(test_loader_20k)}")

Total number of train samples: 1747933
Total number of test samples: 682084
Total number of (train) batches: 782
Total number of (test) batches: 157


In [93]:
class HLSTMCNN(nn.Module):
    def __init__(self, num_classes=101):
        super(HLSTMCNN, self).__init__()
        self.resnet = resnet50(pretrained=True)
        self.resnet.fc = nn.Sequential(nn.Linear(self.resnet.fc.in_features, 300))
        self.lstm = nn.LSTM(input_size=300, hidden_size=256, num_layers=3)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x_3d):
        hidden = None

        # Iterate over each frame of a video in a video of batch * frames * channels * height * width
        for t in range(x_3d.size(1)):
            with torch.no_grad():
                x = self.resnet(x_3d[:, t])
            # Pass latent representation of frame through lstm and update hidden state
            out, hidden = self.lstm(x.unsqueeze(0), hidden)

        # Get the last hidden state (hidden is a tuple with both hidden and cell state in it)
        x = self.fc1(hidden[0][-1])
        x = F.relu(x)
        x = self.fc2(x)

        return x

In [94]:
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.0.1+cu117
True
NVIDIA GeForce RTX 3090


In [95]:
device = torch.device("cuda")  # use gpu if available
target_model = HLSTMCNN().to(device=device)
optimiser=torch.optim.SGD(target_model.parameters(),lr=0.01,momentum=0.9)
cost = torch.nn.CrossEntropyLoss()

In [103]:
def target_train(train_loader, target_model, optimiser):
    target_model.train()
    size = len(train_loader.dataset)
    correct = 0
    total_loss=[]
    for batch, (X, Y) in enumerate(tqdm(train_loader)):
        X, Y = X.to(device=device), Y.to(device=device)
        #print(X.shape, Y.shape)
        optimiser.zero_grad()
        pred = target_model(X)
        #print(pred, Y)
        #pred = pred.flatten()
        loss = cost(pred, Y)
        loss.backward()
        optimiser.step()
        _, output = torch.max(pred, 1)
        #print(output, Y)
        correct+= (output == Y).sum().item()
        total_loss.append(loss.item())
        #batch_count+=batch
        #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()

    #print(correct, size)
    correct /= size
    loss= sum(total_loss)/(batch+1)
    result_train=100*correct
    torch.save(target_model, './base_model_100k.pt')

    print(f'\nTraining Performance:\nacc: {(100*correct):>0.1f}%, avg loss: {loss:>8f}\n')

    return loss, result_train


In [104]:
from sklearn.ensemble import IsolationForest
def target_train_with_outlier_detection(train_loader, target_model, optimiser, val_out=0.05):
   target_model.train()
   size = len(train_loader.dataset)
   correct = 0
   total_loss=[]
   total_sample=0
   for batch, (X, Y) in enumerate(tqdm(train_loader)):
       X, Y = X.to(device=device), Y.to(device=device)
       #print(X.shape, Y.shape)
       X_0_1=X[:,0, 0, :,:]
       X_0_1 = X_0_1.reshape(X.shape[0], X_0_1.shape[1]*X_0_1.shape[2])/255.0
       #print(X[0][0])

       isf = IsolationForest(contamination=val_out, n_jobs=-1)
       X_0_1=X_0_1.cpu().detach().numpy()
       _ = isf.fit(X_0_1)


       # Predictions
       preds = isf.predict(X_0_1)
       non_outl=X[preds!=-1]
       new_X=non_outl
    #    print(non_outl.shape)
       #new_X[batch*batch_size_train + len(non_outl)-1]=new_X
    #    print(f'Batch {batch} >> {len(non_outl)} Non-Outliers detected with contamination {val_out}')
       outl_lab = Y[preds==-1]
       nonoutl_lab = Y[preds!=-1]
       new_Y=nonoutl_lab
       total_sample=total_sample+new_X.shape[0]
       optimiser.zero_grad()
       pred = target_model(new_X)
       #print(pred, Y)
       #pred = pred.flatten()
       loss = cost(pred, new_Y)
       loss.backward()
       optimiser.step()
       _, output = torch.max(pred, 1)
       #print(output, Y)
       correct+= (output == new_Y).sum().item()
       total_loss.append(loss.item())
       #batch_count+=batch
       #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()


   #print(correct, size)
   correct /= size
   loss= sum(total_loss)/(batch+1)
#    print("Total Sample: ", total_sample)
   result_train=100*correct
   torch.save(target_model, '../models/base_model_100k_OD.pt')


   print(f'\nTraining Performance:\nacc: {(100*correct):>0.1f}%, avg loss: {loss:>8f}\n')
  
   return loss, result_train


In [105]:
def target_utility(test_loader, target_model, batch_size = batch_size):
    size = len(test_loader.dataset)
    target_model.eval()
    test_loss, correct = 0, 0
    correct = 0
    total=0
    counter_a=0
    #with torch.no_grad():
    for batch, (X, Y) in enumerate(tqdm(test_loader)):
        X, Y = X.to(device=device), Y.to(device=device)
        X.requires_grad = True
        pred = target_model(X)
        #print("Y is: ", Y)
        counter_a=counter_a+1
        #test_loss += cost(pred, Y).item()
        total += Y.size(0)
        # Forward pass the data through the model
        _, output_res = torch.max(pred, -1)
        #print("pred is: ",output_res)
        correct += ((output_res) == Y).sum().item()


    # Calculate final accuracy
    final_acc = correct/float(total)
    print(f"Target Model Accuracy = {correct} / {total} = {final_acc}")

    # Return the accuracy
    return final_acc



In [106]:
def target_utility_top5(test_loader, target_model, batch_size = batch_size):
   size = len(test_loader.dataset)
   target_model.eval()
   test_loss, correct = 0, 0
   correct = 0
   topk_correct=0
   total=0
   counter_a=0
   #with torch.no_grad():
   for batch, (X, Y) in enumerate(tqdm(test_loader)):
       X, Y = X.to(device=device), Y.to(device=device)
       X.requires_grad = True
       pred = target_model(X)
       #print("Y is: ", Y)
       counter_a=counter_a+1
       #test_loss += cost(pred, Y).item()
       #correct += (pred.argmax(1)==Y).type(torch.float).sum().item()
       #data, target = data.to(device), target.to(device)
       # Set requires_grad attribute of tensor. Important for Attack
       total += Y.size(0)
       # Forward pass the data through the model
       _, output_res = torch.max(pred, -1)
       #print("pred is: ",output_res)
       correct += ((output_res) == Y).sum().item()
       top_k=5
       for test_sample in range(X.shape[0]):
           val, idx=torch.topk(pred[test_sample, :], top_k)
           for check in range(top_k):
               if (idx[check]==Y[test_sample]):
                   topk_correct+=1


   # Calculate final accuracy for this epsilon
   topk_acc= topk_correct/float(total)
   final_acc = correct/float(total)
   print(f"Target Model Top-1 Acc = {correct} / {total} = {final_acc}, Top-5 Acc= {topk_correct} / {total}={topk_acc}")


   # Return the accuracy and an adversarial example
   return final_acc, topk_acc



In [107]:
import smtplib
from email.mime.text import MIMEText

def send_email(subject, message):
    sender_email = 'cameronkeithgolf@gmail.com'
    receiver_email = 'cameronkeithgolf@gmail.com'
    smtp_server = 'smtp.gmail.com'
    smtp_port = 587
    smtp_username = 'cameronkeithgolf@gmail.com'
    smtp_password = 'lkhauhjcqytyrarr'

    msg = MIMEText(message)
    msg['From'] = sender_email
    msg['Subject'] = subject
    msg['To'] = receiver_email

    server = smtplib.SMTP(smtp_server, smtp_port)
    server.starttls()
    server.login(smtp_username, smtp_password)
    server.sendmail(sender_email, receiver_email, msg.as_string())
    server.quit()


In [ ]:
target_epochs=15
loss_train_tr, loss_test_tr=[],[]
for t in tqdm(range(target_epochs)):
    print(f'Epoch {t+1}\n-------------------------------')
    print("+++++++++Target Training Starting+++++++++")
    tr_loss, result_train=target_train(train_loader_100k, target_model, optimiser)
    loss_train_tr.append(tr_loss)

    final_acc, topk_acc=target_utility_top5(test_loader_20k, target_model, batch_size = batch_size)

    print(f"train loss: {tr_loss}")
    print(f"train accuracy: {result_train}")
    print(f"Top-1 Test Acc: {final_acc}, Top-5 Test Acc: {topk_acc}")

    send_email(
    f'CNN Training update after {t+1} epochs',
    f'''
        Train loss: {tr_loss}
        Train accuracy: {result_train}
        Top-1 Test accuracy: {final_acc}
        Top-5 Test accuracy: {topk_acc}
        '''
    )


In [ ]:
print("+++++++++Target Test+++++++++")

final_acc=target_utility(test_loader_20k, target_model, batch_size = batch_size)

print(f"Test Acc: {final_acc}")                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

+++++++++Target Test+++++++++


100%|██████████| 196/196 [20:33<00:00,  6.30s/it]

Target Model Accuracy = 14652 / 25000 = 0.58608
Test Acc: 0.58608
